In [16]:
import pandas as pd 
import ast
import json

In [17]:
data = pd.read_csv('datasets/raw/movie_tmdb_ajout.csv' , sep=',')
movie = pd.read_csv('datasets/raw/movie.csv', sep=',')

# Convertir les chaînes en listes de dictionnaires
data['spoken_languages'] = data['spoken_languages'].apply(ast.literal_eval)
# Convertir la chaîne en liste python avec ast.literal_eval (plus sûr que json.loads si la chaîne n'est pas en format JSON strict)
data['production_countries'] = data['production_countries'].apply(ast.literal_eval)


# Trier les genres qu'il nous faut
# Trier le spoken_language en fr ou French ou Fr
# Changer le format de la realese_date en date
# Changer le nom de imdb_id en tconst
# Rajouter une colonne décennie
# Rajouter les informations sur les acteurs, actrices et directeurs avec l'api
# Trier sur les meilleurs films top 5 %  des films par décennie
# Voir lequel des vite_average et vote_count gardé
# Merger avec movie_csv
# Détermination des types de colonnes 
# data_fr['spoken_languages'].apply(type).value_counts()
# Détermination des types de colonnes 
# movie_tmdb['production_countries'].apply(type).value_counts()

In [18]:
#convertir les colonnes (genres, actors, actresses, directors) en liste
movie['genres'] = movie['genres'].apply(lambda x: x.split(',') if isinstance(x, str) else x)
movie['actors'] = movie['actors'].apply(lambda x: x.split(',') if isinstance(x, str) else x)
movie['actresses'] = movie['actresses'].apply(lambda x: x.split(',') if isinstance(x, str) else x)
movie['directors'] = movie['directors'].apply(lambda x: x.split(',') if isinstance(x, str) else x)

In [19]:
data_fr = data.copy()

# Filtrer pour ne garder que les films exclusivement en français
data_fr = data_fr[data_fr['spoken_languages'].apply(lambda langs: any(lang.get('iso_639_1') == 'fr' for lang in langs))]

# Supprimer les colonnes inutiles
data_fr = data_fr.drop(columns=['adult','backdrop_path','belongs_to_collection','budget', 'homepage','production_companies',
                                'revenue','origin_country','tagline','video','title'])

""" Realease date formatage et création de 2 colonnes"""
#Changement du format de la released_date 
data_fr['release_date'] = pd.to_datetime(data_fr['release_date'])

# Rajout de la colonne decennie et start year
data_fr['startYear'] = data_fr['release_date'].dt.year

# Créer la colonne décennie
data_fr['decennie'] = (data_fr['startYear'] // 10) * 10

""" Genre Formatage """
# Convertir les chaînes en listes de dictionnaires
data_fr['genres'] = data_fr['genres'].apply(ast.literal_eval)

# Liste des genres à exclure
genres_exclus = {'Documentaire', 'Musique', 'Téléfilm'}

movie_tmdb = data_fr.copy() 

# Filtrer pour supprimer les films contenant ces genres
movie_tmdb = movie_tmdb[~movie_tmdb['genres'].apply(lambda genre_list: any(g['name'] in genres_exclus for g in genre_list))]

# Changement du nom imdb_id en tconst
movie_tmdb.rename(columns={'imdb_id': 'tconst'}, inplace=True)

""" Réinitialisation de l'index """
# Réinitialiser les index
movie_tmdb.reset_index(drop=True, inplace=True)

# Affichage
movie_tmdb

,genres,id,tconst,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,runtime,spoken_languages,status,vote_average,vote_count,actors,actresses,directors,startYear,decennie
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",575265,tt9603208,en,Mission: Impossible - The Final Reckoning,Ethan Hunt se rend à Londres avec son équipe d...,267.7424,/sOvTMbTHmp13LWYvgMXDvbY1auX.jpg,"[{'iso_3166_1': 'US', 'name': 'United States o...",2025-05-15,163,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,7.100,179,"['Tom Cruise', 'Ving Rhames', 'Simon Pegg', 'E...","['Hayley Atwell', 'Pom Klementieff', 'Janet Mc...",['Christopher McQuarrie'],2025,2020
1,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",1144430,tt29768333,fr,Balle perdue 3,"Dans un dernier opus chargé en adrénaline, le ...",145.5038,/xccJs9vGCEvch1H9SIMSjNqrLKU.jpg,"[{'iso_3166_1': 'BE', 'name': 'Belgium'}, {'is...",2025-05-06,111,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,6.799,127,"['Alban Lenoir', 'Nicolas Duvauchelle', 'Gérar...","['Stéfi Celma', 'Pascale Arbillot', 'Anne Serra']",['Guillaume Pierret'],2025,2020
2,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na...",939243,tt18259086,en,Sonic the Hedgehog 3,"Sonic, Knuckles et Tails sont à nouveau réunis...",73.6897,/gERwLGTa6JGN4qXjkip13eDaxy1.jpg,"[{'iso_3166_1': 'US', 'name': 'United States o...",2024-12-19,100,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,7.715,2587,"['Jim Carrey', 'Ben Schwartz', 'Keanu Reeves',...","[""Colleen O'Shaughnessey"", 'Tika Sumpter', 'Kr...",['Jeff Fowler'],2024,2020
3,"[{'id': 16, 'name': 'Animation'}, {'id': 18, '...",1064486,tt23770030,en,Memoir of a Snail,"La vie de Grace Pudel, petite fille solitaire,...",34.0607,/oYpq6XFoubsezCt1o28TwnraD0a.jpg,"[{'iso_3166_1': 'AU', 'name': 'Australia'}]",2024-10-17,94,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,8.068,505,"['Kodi Smit-McPhee', 'Dominique Pinon', 'Tony ...","['Sarah Snook', 'Jacki Weaver', 'Magda Szubans...",['Adam Elliot'],2024,2020
4,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",1087891,tt0899043,en,The Amateur,"Un cryptographe de la CIA qui, après la mort t...",27.0158,/l2EmTzKN7qZvAW6p7JfH6v09tAO.jpg,"[{'iso_3166_1': 'US', 'name': 'United States o...",2025-04-09,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,6.651,259,"['Rami Malek', 'Holt McCallany', 'Danny Sapani...","['Rachel Brosnahan', 'Julianne Nicholson', 'Ca...",['James Hawes'],2025,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,"[{'id': 18, 'name': 'Drame'}, {'id': 10749, 'n...",1001249,tt29301256,fr,Le système Victoria,"Directeur de travaux, David est à la tête du...",0.2427,/jecs6kdzEyks717OJtARIitZAQ6.jpg,"[{'iso_3166_1': 'FR', 'name': 'France'}]",2025-03-05,101,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,6.600,5,"['Damien Bonnard', 'Sharif Andoura', 'Cédric A...","['Jeanne Balibar', 'Antonia Buresi']",['Sylvain Desclous'],2025,2020
432,"[{'id': 35, 'name': 'Comédie'}, {'id': 18, 'na...",892557,tt32892130,fr,Ma Famille chérie,Une célébre chanteuse d’opéra à la retraite re...,0.3326,/xVJR37M9YJ2nXxRP2dvOxFhZJn1.jpg,"[{'iso_3166_1': 'FR', 'name': 'France'}]",2024-08-11,83,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,5.500,4,"['Sam Spruell', 'Stefano Cassetti', 'Élie Semo...","['Marisa Berenson', 'Laëtitia EÏDO', 'Jeanne B...",['Isild Le Besco'],2024,2020
433,"[{'id': 18, 'name': 'Drame'}, {'id': 35, 'name...",1313226,tt33400055,fr,Kika,"Alors que son compagnon vient de mourir, Kika ...",0.1536,/eYjk3arB4qEc1r5dHGbjG1oihUT.jpg,"[{'iso_3166_1': 'BE', 'name': 'Belgium'}, {'is...",2025-11-12,108,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,0.000,0,"['Makita Samba', 'Thomas Coumans', 'Bernard Bl...","['Manon Clavel', 'Anaël Snoek', 'Kadija Leclère']",['Alexe Poukine'],2025,2020
434,"[{'id': 18, 'name': 'Drame'}]",1262533,tt16893002,fr,Le silence d

In [20]:
movie_tmdb['genres'] = movie_tmdb['genres'].apply(
    lambda genre_list: [g.get('name') for g in genre_list if isinstance(g, dict) and g.get('name')]
)

movie_tmdb['spoken_languages'] = movie_tmdb['spoken_languages'].apply(
    lambda langs: [l.get('iso_639_1') for l in langs if isinstance(l, dict) and l.get('iso_639_1')]
)

movie_tmdb['production_countries'] = movie_tmdb['production_countries'].apply(lambda countries: [c['name'] for c in countries])
movie_tmdb

,genres,id,tconst,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,runtime,spoken_languages,status,vote_average,vote_count,actors,actresses,directors,startYear,decennie
0,"[Action, Aventure, Thriller]",575265,tt9603208,en,Mission: Impossible - The Final Reckoning,Ethan Hunt se rend à Londres avec son équipe d...,267.7424,/sOvTMbTHmp13LWYvgMXDvbY1auX.jpg,[United States of America],2025-05-15,163,"[en, fr, iu]",Released,7.100,179,"['Tom Cruise', 'Ving Rhames', 'Simon Pegg', 'E...","['Hayley Atwell', 'Pom Klementieff', 'Janet Mc...",['Christopher McQuarrie'],2025,2020
1,"[Action, Crime, Thriller, Drame]",1144430,tt29768333,fr,Balle perdue 3,"Dans un dernier opus chargé en adrénaline, le ...",145.5038,/xccJs9vGCEvch1H9SIMSjNqrLKU.jpg,"[Belgium, France]",2025-05-06,111,[fr],Released,6.799,127,"['Alban Lenoir', 'Nicolas Duvauchelle', 'Gérar...","['Stéfi Celma', 'Pascale Arbillot', 'Anne Serra']",['Guillaume Pierret'],2025,2020
2,"[Action, Science-Fiction, Comédie, Familial]",939243,tt18259086,en,Sonic the Hedgehog 3,"Sonic, Knuckles et Tails sont à nouveau réunis...",73.6897,/gERwLGTa6JGN4qXjkip13eDaxy1.jpg,"[United States of America, Japan]",2024-12-19,100,"[en, fr, ja, es]",Released,7.715,2587,"['Jim Carrey', 'Ben Schwartz', 'Keanu Reeves',...","[""Colleen O'Shaughnessey"", 'Tika Sumpter', 'Kr...",['Jeff Fowler'],2024,2020
3,"[Animation, Drame, Comédie]",1064486,tt23770030,en,Memoir of a Snail,"La vie de Grace Pudel, petite fille solitaire,...",34.0607,/oYpq6XFoubsezCt1o28TwnraD0a.jpg,[Australia],2024-10-17,94,"[fr, en]",Released,8.068,505,"['Kodi Smit-McPhee', 'Dominique Pinon', 'Tony ...","['Sarah Snook', 'Jacki Weaver', 'Magda Szubans...",['Adam Elliot'],2024,2020
4,"[Thriller, Action]",1087891,tt0899043,en,The Amateur,"Un cryptographe de la CIA qui, après la mort t...",27.0158,/l2EmTzKN7qZvAW6p7JfH6v09tAO.jpg,[United States of America],2025-04-09,123,"[en, fr, ru]",Released,6.651,259,"['Rami Malek', 'Holt McCallany', 'Danny Sapani...","['Rachel Brosnahan', 'Julianne Nicholson', 'Ca...",['James Hawes'],2025,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,"[Drame, Romance, Comédie]",1001249,tt29301256,fr,Le système Victoria,"Directeur de travaux, David est à la tête du...",0.2427,/jecs6kdzEyks717OJtARIitZAQ6.jpg,[France],2025-03-05,101,[fr],Released,6.600,5,"['Damien Bonnard', 'Sharif Andoura', 'Cédric A...","['Jeanne Balibar', 'Antonia Buresi']",['Sylvain Desclous'],2025,2020
432,"[Comédie, Drame]",892557,tt32892130,fr,Ma Famille chérie,Une célébre chanteuse d’opéra à la retraite re...,0.3326,/xVJR37M9YJ2nXxRP2dvOxFhZJn1.jpg,[France],2024-08-11,83,"[en, fr, it]",Released,5.500,4,"['Sam Spruell', 'Stefano Cassetti', 'Élie Semo...","['Marisa Berenson', 'Laëtitia EÏDO', 'Jeanne B...",['Isild Le Besco'],2024,2020
433,"[Drame, Comédie]",1313226,tt33400055,fr,Kika,"Alors que son compagnon vient de mourir, Kika ...",0.1536,/eYjk3arB4qEc1r5dHGbjG1oihUT.jpg,"[Belgium, France]",2025-11-12,108,[fr],Released,0.000,0,"['Makita Samba', 'Thomas Coumans', 'Bernard Bl...","['Manon Clavel', 'Anaël Snoek', 'Kadija Leclère']",['Alexe Poukine'],2025,2020
434,[Drame],1262533,tt16893002,fr,Le silence de Sibel,"Août 2014, à Sinjar, au nord-ouest de l’Irak, ...",0.1614,/7FrbBwDFaKS8vB7iZ2lwt23LY3g.jpg,[France],2024-05-01,98,[fr],Released,5.000,1,[],"['Laëtitia EÏDO', 'Marie-Bénédicte Cazeneuve']",['Aly Yeganeh'],2024,2020


In [21]:
# Changement du nom pour faciliter la concaténation
movie.rename(columns={'decennie_x': 'decennie'}, inplace=True)
movie_tmdb.rename(columns={'runtime': 'runtimeMinutes'}, inplace=True)

# Concaténation et non pas une merge car on rajoute des lignes en plus afin d'enrichir notre dataframe
films =  pd.concat([movie, movie_tmdb], ignore_index=True)
films.reset_index(drop=True, inplace=True)
films

# Suppression des doublons
films.drop_duplicates(subset='id', inplace=True)

# Reset de l'index 
films.reset_index(drop=True, inplace=True)
films

,tconst,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,spoken_languages,vote_count,...,numVotes,startYear,runtimeMinutes,genres,actors,actresses,directors,id,status,vote_average
0,tt0055747,fr,L'Amour à vingt ans,Love at Twenty unites five directors from five...,3.7700,/aup2QCYCsyEeQfpboXy0f4uj8aE.jpg,"['DE', 'FR', 'IT', 'JP', 'PL']",1962-06-22,"['it', 'ja', 'pl', 'fr', 'de']",41,...,2515.0,1962.0,110.0,"[Drama, Romance]","[Jean-Pierre Léaud, Patrick Auffay, François...","[Marie-France Pisier, Rosy Varte, Cristina G...","[Shintarô Ishihara, Marcel Ophüls, Renzo Ros...",NaN,NaN,NaN
1,tt9603208,en,Mission: Impossible - The Final Reckoning,Ethan Hunt se rend à Londres avec son équipe d...,267.7424,/sOvTMbTHmp13LWYvgMXDvbY1auX.jpg,[United States of America],2025-05-15 00:00:00,"[en, fr, iu]",179,...,NaN,2025.0,163.0,"[Action, Aventure, Thriller]","['Tom Cruise', 'Ving Rhames', 'Simon Pegg', 'E...","['Hayley Atwell', 'Pom Klementieff', 'Janet Mc...",['Christopher McQuarrie'],575265.0,Released,7.100
2,tt29768333,fr,Balle perdue 3,"Dans un dernier opus chargé en adrénaline, le ...",145.5038,/xccJs9vGCEvch1H9SIMSjNqrLKU.jpg,"[Belgium, France]",2025-05-06 00:00:00,[fr],127,...,NaN,2025.0,111.0,"[Action, Crime, Thriller, Drame]","['Alban Lenoir', 'Nicolas Duvauchelle', 'Gérar...","['Stéfi Celma', 'Pascale Arbillot', 'Anne Serra']",['Guillaume Pierret'],1144430.0,Released,6.799
3,tt18259086,en,Sonic the Hedgehog 3,"Sonic, Knuckles et Tails sont à nouveau réunis...",73.6897,/gERwLGTa6JGN4qXjkip13eDaxy1.jpg,"[United States of America, Japan]",2024-12-19 00:00:00,"[en, fr, ja, es]",2587,...,NaN,2024.0,100.0,"[Action, Science-Fiction, Comédie, Familial]","['Jim Carrey', 'Ben Schwartz', 'Keanu Reeves',...","[""Colleen O'Shaughnessey"", 'Tika Sumpter', 'Kr...",['Jeff Fowler'],939243.0,Released,7.715
4,tt23770030,en,Memoir of a Snail,"La vie de Grace Pudel, petite fille solitaire,...",34.0607,/oYpq6XFoubsezCt1o28TwnraD0a.jpg,[Australia],2024-10-17 00:00:00,"[fr, en]",505,...,NaN,2024.0,94.0,"[Animation, Drame, Comédie]","['Kodi Smit-McPhee', 'Dominique Pinon', 'Tony ...","['Sarah Snook', 'Jacki Weaver', 'Magda Szubans...",['Adam Elliot'],1064486.0,Released,8.068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,tt29301256,fr,Le système Victoria,"Directeur de travaux, David est à la tête du...",0.2427,/jecs6kdzEyks717OJtARIitZAQ6.jpg,[France],2025-03-05 00:00:00,[fr],5,...,NaN,2025.0,101.0,"[Drame, Romance, Comédie]","['Damien Bonnard', 'Sharif Andoura', 'Cédric A...","['Jeanne Balibar', 'Antonia Buresi']",['Sylvain Desclous'],1001249.0,Released,6.600
433,tt32892130,fr,Ma Famille chérie,Une célébre chanteuse d’opéra à la retraite re...,0.3326,/xVJR37M9YJ2nXxRP2dvOxFhZJn1.jpg,[France],2024-08-11 00:00:00,"[en, fr, it]",4,...,NaN,2024.0,83.0,"[Comédie, Drame]","['Sam Spruell', 'Stefano Cassetti', 'Élie Semo...","['Marisa Berenson', 'Laëtitia EÏDO', 'Jeanne B...",['Isild Le Besco'],892557.0,Released,5.500
434,tt33400055,fr,Kika,"Alors que son compagnon vient de mourir, Kika ...",0.1536,/eYjk3arB4qEc1r5dHGbjG1oihUT.jpg,"[Belgium, France]",2025-11-12 00:00:00,[fr],0,...,NaN,2025.0,108.0,"[Drame, Comédie]","['Makita Samba', 'Thomas Coumans', 'Bernard Bl...","['Manon Clavel', 'Anaël Snoek', 'Kadija Leclère']",['Alexe Poukine'],1313226.0,Released,0.000
435,tt16893002,fr,Le silence de Sibel,"Août 2014, à Sinjar, au nord-ouest de l’Irak, ...",0.1614,/7FrbBwDFaKS8vB7iZ2lwt23LY3g.jpg,[France],2024-05-01 00:00:00,[fr],1,...,NaN,2024.0,98.0,[Drame],[],"['Laëtitia EÏDO', 'Marie-Bénédicte Cazeneuve']",['Aly Yeganeh'],1262533.0,Released,5.000
